In [1]:
import requests
import os
from twilio.rest import Client

#Link - https://www.twilio.com/try-twilio
#Create a free account and grab all the relevant informationrelevant information - endpoint, account_sid, auth_token, appid, etc.
endpoint = ''
account_sid = ''
auth_token = '' 

weather_parameters = {
    #https://www.latlong.net/ - lat/long coordinates
    #city - chicago - lat/long - 41.878113/-87.629799
    'lat': '41.878113',
    'lon': '-87.629799',
    'appid': '',
    #website - https://openweathermap.org/api/one-call-api
    #note - it should be a comma-delimited list (without spaces)
    'exclude': 'current,minutely,daily'
}

response = requests.get(endpoint, params = weather_parameters)
#exception
response.raise_for_status()
#json viewer - http://jsonviewer.stack.hu/
#print(response.json())
weather_data = response.json()
#print(weather_data)
#weather services - id

weather_hourly_data = weather_data['hourly'][:12]
is_whatever = False

for hourly_data in weather_hourly_data:
    #[0] - index = 0 - grabs the first item in the list
    data = hourly_data['weather'][0]['id']
    if int(data) < 804:
        is_whatever = True

if is_whatever:
    #programmable sms python quickstart - https://www.twilio.com/docs/sms/quickstart/python
    #send an sms using twilio - programming block ex. - same format was used
    client = Client(account_sid, auth_token)
    message = client.messages \
        .create(
        body = 'Go to https://openweathermap.org/city/4887398 and look at the weather forecast for Chicago. There is a likelihood of it raining today. Go grab your ☔️.',
        from_ = '',
        to = ''
    )
    print(message.status)   

MissingSchema: Invalid URL '': No schema supplied. Perhaps you meant http://?